# Probability and Statistics

Probability distributions and statistical arguments are of fundamental importance in all branches of physics. Examples include: 
- **Statistical Mechanics** probability distributions are used to conceptualize and describe the aggregate behavior of large numbers of particles (molecules in a gas) where it would it be impractical to track the behavior of each individual particle.  

- **Quantum Mechanics** probability distributions describe the likelihood of finding a particle in a particular state.

- **Astrophysics** All structure in the universe is the result of the interplay between gravity and the initial conditions of the universe. The Universe's large scale structure and its initial conditions are described with probability distributions. Statistical methods are used to measure, analyze, and interpret the distribution of matter in the Universe.

- **Particle Physics** Experiments involving particle collisions use statistical methods to interpret and analyze data. Probability is crucial in the identification of particles and the verification of new particle discoveries.

- **Nuclear Physics** Nuclear reactions and decay processes are described using probability distributions.

- **Condensed Matter Physics** Statistical methods are used to describe the behavior of large numbers of particles in a solid or liquid.

- **Experimental Physics and Observational Astronomy** Statistical methods are used to analyze data from experiments/observations, determine whether physical models are consistent with such data, and to determine the significance of experimental/observational results.

## Probability Distributions

In the last homework you analyzed household income data from the [American Community Survey](https://www.census.gov/programs-surveys/acs) (ACS) conducted by the United States Census Bureau. Using this data we constructed and plotted histograms of household income, as shown below. 

In [ ]:
from census_utils import plot_census_data, generate_fake_census_data
plot_census_data()

The histogram above shows the distribution of household income in the United States in 2022. The data is binned into bins of width $\Delta I = \$10,000$ from $\$0$ to $\$300,000$, and the histogram indicates the percentage of households in each bin. To make the high-income *tail* of this distribution easier to visualize, I've chosen the last two bins to be wider: the second to last bin is from $\$300,000$ to $\$350,000$, and the last bin includes all incomes larger than $\$350,000$. 

The input data for this histogram is a list of incomes, $i_j$ for the $N_{\rm tot} = 127,970,381$ households in the United States (with income $> 0$), where $j=1,2,3,..., N_{\rm tot}$ (the average size of a US household is 2.3 people, according to this dataset).   Here $i_j$ is a continuous variable that can take on any value, whereas the centers of the histogram bins, which we will denote as $I_k$, are discrete. 

Let's  compute the histogram  over the full range of the data (i.e. without the two wider bins at the end) and use it to frame our discussion of proability distributions. You know from the homework that the real data was not a list of incomes for all $127,970,381$ housholds in the United States, but rather a shorter list of records with weights that indicate the number of households that each record represents. For the purposes of the discussion here, I've 
written code to generate a fake list of $N_{\rm tot}= 1,000,000$ incomes (without the weights) drawn from the distribution of the real data.

In [ ]:
import numpy as np
incomes = generate_fake_census_data()
N_tot = len(incomes)
print(N_tot)

In [ ]:
from matplotlib import pyplot as plt
# Construct the bins
bin_width = 10000
# Maximum value is 2,481,200, so go up to 2.5 million
I_edges = np.arange(0, 2500000, bin_width)

# Compute the histogram using numpy
N_of_I, _ = np.histogram(incomes, bins=I_edges)
I_centers = (I_edges[1:] + I_edges[:-1])/2

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(I_edges[:-1], N_of_I, width=np.diff(I_edges), align='edge', color='orange', 
       alpha=1.0, edgecolor='black', zorder=10, label='Census data')
plt.xlabel('I (income)', fontsize=16)
plt.ylabel('N(I) (number per bin)', fontsize=16)
plt.legend()
plt.yscale('log')

Each histogram bin $k$ contains the number of households, $N(I_k)$, with income $i_j$ in the range $I_k - \Delta I/2 \leq i_j < I_k + \Delta I/2$, where $I_k$ is the center of the bin. The total number of households is then
<a id='eqn:N_tot'></a>
$$
N_{\rm tot} = \sum_{k=0}^{N_{\rm bins}-1} N(I_k) \tag{1},
$$
where $N_{\rm bins}$ is the number of bins in the histogram.

In [ ]:
# Double check that our histogram includes all the data
assert np.sum(N_of_I) == N_tot  # an assertion error will be raised if the condition is not met
print('N_tot:', N_tot)

### Mean, Variance, and Mode

The mean income $\langle i \rangle$ is the average income over all households, 
$$
\langle i \rangle = \frac{1}{N_{\rm tot}} \sum_{j=0}^{N_{\rm tot-1}} i_j \approx \frac{1}{N_{\rm tot}} \sum_{k=0}^{N_{\rm bins}-1} I_k N(I_k) = \langle I\rangle,
$$
where the second equality is an approximation since $i_j$ is a continuous quantity, whereas the $I_k$ are discrete. In the limit that $\Delta I \rightarrow 0$, the approximation becomes exact and $\langle i \rangle = \langle I\rangle$.

In [ ]:
# Two ways to compute the mean
i_bar = np.sum(incomes)/N_tot # see also np.mean(incomes)
I_bar = np.sum(N_of_I*I_centers)/N_tot
print(f'<i> = ${i_bar:.2f} ; <I> = ${I_bar:.2f}')

We define the relative frequency (probability) of a household having a given income $I_k$ as
$$
P(I_k) = \frac{N(I_k)}{N_{\rm tot}}. 
$$
Note that the probability distribution or distribution function, $P(I_k)$, satisfies the **normalization condition**
$$
\sum_{k=0}^{N_{\rm bins}-1} P(I_k) = \frac{1}{N_{\rm tot}} \sum_{k=0}^{N_{\rm bins}-1} N(I_k) = 1, 
$$
where we have used eqn.(<a href="#eqn:N_tot">1</a>).

In [ ]:
P_of_I = N_of_I/N_tot
assert np.isclose(np.sum(P_of_I), 1.0)
print(f'The sum of the probabilities is: {np.sum(P_of_I)}')

So in terms of $P(I_k)$, the mean income is
$$
\langle I\rangle = \sum_{k=0}^{N_{\rm bins}-1} I_k P(I_k).
$$

We can also calculate the mean-square income, $\langle i^2\rangle$, which is the average of the square of the income over all households,
$$
\langle i^2\rangle = \frac{1}{N_{\rm tot}}\sum_{j=0}^{N_{\rm tot}-1} i_j^2 \approx \sum_{k=0}^{N_{\rm bins}-1} I_k^2 P(I_k) = \langle I^2\rangle,
$$
or taking the square-root, we obtain the **root-mean-square** (RMS) income,
$$
I_{\rm rms} = \sqrt{\langle I^2\rangle}.
$$

A related useful quantity is the **variance** of the income distribution, which is a measure of the spread of the distribution. The variance is defined as
$$
\sigma_i^2 \equiv \langle (i - \langle i\rangle)^2\rangle \approx  \langle (I - \langle I\rangle)^2\rangle \equiv \sigma_I^2 = \sum_{k=0}^{N_{\rm bins}-1} (I_k - \langle I\rangle)^2 P(I_k), 
$$
where again the approximation arises because $i_j$ is continuous and $I_k$ is discrete.

Note that by expanding the square in the definition of the variance, we can write
$$
\begin{align*}
{\rm Var}(i) = \sigma_i^2 & \approx \sigma_I^2 = \sum_{k=0}^{N_{\rm bins}-1} (I_k^2 - 2 \langle I\rangle I_k + \langle I\rangle^2) P(I_k)\\
         &  = \sum_{k=0}^{N_{\rm bins}-1} I_k^2 P(I_k) - 2 \langle I\rangle \sum_{k=0}^{N_{\rm bins}-1} I_k P(I_k) + \langle I\rangle^2 \sum_{k=0}^{N_{\rm bins}-1} P(I_k)\\
        & = \langle I^2\rangle - 2\langle I\rangle^2 + \langle I\rangle^2\\
        &  = \langle I^2\rangle - \langle I\rangle^2.
\end{align*}
$$

The **standard deviation** is the square root of the variance, 
$$
\sigma_I = \sqrt{\sigma_I^2} = \sqrt{\langle I^2\rangle - \langle I\rangle^2}.
$$

In [ ]:
# Compute the probability distribution P_of_I
P_of_I = (N_of_I/N_tot)
# Two ways to compute the standard deviation 
sigma_i = np.std(incomes) # from the original data
sigma_i_2 = np.sqrt(np.mean((incomes - i_bar)**2)) # from the original data
var_i = sigma_i**2
sigma_I = np.sqrt(np.sum(P_of_I*(I_centers - I_bar)**2)) # from the histogram
assert np.isclose(sigma_i, sigma_i_2)
print(f'sigma_i = ${sigma_i:.2f} ; sigma_I = ${sigma_I:.2f}')


Another useful quantity for characterizing probability distributions is the **mode**
$$
I_{\rm mode} = \max_k P(I_k),
$$
which is the value of $I_k$ at which the probability distribution is maximized.  This is the most probable value of the income if you randomly draw households from the US population. 

In [ ]:
# Compute the mode
print(np.max(P_of_I))
k_max = np.argmax(P_of_I)
print(k_max)
I_mode = I_centers[k_max]
print(f'The mode of the distribution is: ${I_mode:.2f}')

### Cumulative Distribution Function and Median

Another very important quantity is the **cumulative distribution function** (CDF), which is defined by 
$$
{\rm CDF}(\le I_k) = \sum_{k^\prime=0}^{k} P(I_{k^\prime}). 
$$
Since it sums over all $P(I_k)$ for $k^\prime \le k$, the CDF is a measure of the probability that a household has an income less than or equal to $I_k$. For example, for the household income data, if the probability that a household has an income less than or equal to \$128,000 is 75% then ${\rm CDF}(\le \$128,000) = 0.75$. In code, we can compute the CDF using
the `np.cumsum` function in `numpy` as we will show below. 

The **median** is the value of $I_k$ encompassing 50% of the cumulative probability. It is defined by the equation
$$
\sum_{k=0}^{k_{\rm med}} P(I_k) = 0.5 \quad ; \quad I_{\rm k_{\rm med}} \equiv \text{the median income}. 
$$
If we return to the continuous variable $i_j$, the median income is the value of $i_j$ such that 50% of the households have an income less than or equal to this value, which is just the *midpoint* of the list of $i_j$ values when they are sorted in ascending order.

In [ ]:
from matplotlib.ticker import MultipleLocator
# Two ways to compute the median
i_median = np.median(incomes) # First way, from the original data
# Second way, from histogram find the bin that contains 50% of the cumulative probability
P_cumulative = np.cumsum(P_of_I) 
# Cumsum means P_cumulative[0] = P_of_I[0]; P_cumulative[1] = P_of_I[0] + P_of_I[1], etc.
# Find the first bin where the cumulative probability exceeds 0.5
k_med = np.argmax(P_cumulative > 0.5) 
# np.argmax finds the index of the maximum value in a numpy array. For a Boolean array, the
# maximum value is True, and np.argmax defaults to return the first element at which the max 
# is achieved, which is here the first element of the array where the condition is true
I_median = I_centers[k_med]
plt.plot(I_centers, P_cumulative, 'k-')
plt.xlabel('I (income in $)', fontsize=16)
plt.ylabel(r'$CDF(\leq I)$ (cumulative probability)', fontsize=16)
plt.axhline(0.5, color='red', linestyle='--')
plt.plot([i_median, i_median],[0.0, 0.5], color='red', linestyle='--')
# Set the major tick mark spacing on the y-axis to 0.1
plt.gca().yaxis.set_major_locator(MultipleLocator(0.1))
print(f'i_median = ${i_median:.2f} ; I_median = ${I_median:.2f}')


Notice that the mean and median of the US household income distribution are not the same: 

In [ ]:
print(f'The mean=${i_bar:.2f}, whereas the median=${i_median:.2f}')

The reason for this is that the distribution posseses a large tail of high-income households, which pulls the mean towards higher values relative to the median.

### Discrete Versus Continuous Probability Distributions

In some applications the variable of interest is discrete, for example if we are considering rolling a die, there are six discrete possible outcomes: 1, 2, 3, 4, 5, or 6, but a value of 1.5 is impossible. So the probability distribution would similarly have six 
discrete values, $P(1)$, $P(2)$, $P(3)$, $P(4)$, $P(5)$, and $P(6)$.

In other cases the variable is continuous, for example household incomes, $i_j$, can take on any real number in the range $0 \leq i_j < \infty$. In the example above however, we were dealing with a dataset with a finite number of entires. To visualize the probability distribution of a continuous variable for a finite dataset, we use a histogram, $P(I_k)$, with a finite number of discrete bins. 

Now consider the case of a continuous distribution of some variable $x$. We define $P(x)$ by
$$
N(x) = N_{\rm tot} P(x) \Delta x \quad ; \quad N(x)\propto \Delta x,  
$$
where $N(x)$ is the number of events in the range $x$ to $x+\Delta x$. Clearly in the limit $\Delta x \rightarrow 0$ and $N_{\rm tot} \rightarrow \infty$,  $P(x)$ goes from being a histogram to a continuous function 
$$
P(x) = \lim_{\substack{\Delta x \to 0 \\ N_{\rm tot} \to \infty}} \frac{N(x)}{N_{\rm tot} \Delta x}. 
$$

For continuous probability distributions, the discrete sums then become integrals, i.e. 
$$
\int P(x) dx = 1, \quad \quad \text{normalization condition}.
$$
Note that $P(x)$ is a **probability density** and now has units of $[1/x]$, i.e. $P(x)dx$ is the infinitesimal dimensionless probability of finding $x$ in the range $x$ to $x+dx$.

The mean is now defined by
<a id='eqn:mean'></a>
$$
\langle x\rangle = \int x P(x) dx, \quad \quad \text{mean}, \tag{2}
$$
and likewise the variance is
<a id='eqn:variance'></a>
$$
{\rm Var}(x) = \sigma^2 = \int (x - \langle x\rangle)^2 P(x) dx, \quad \quad \text{variance}. \tag{3}
$$

For a continuous variable defined on the entire  real line, $x \in (-\infty, \infty)$, the CDF then becomes
$$
{\rm CDF}(\le x) = \int_{-\infty}^x P(x^\prime) dx^\prime,
$$
where again ${\rm CDF}(\le x)$ is the probability of encountering a value that is less than or equal to $x$.  

The median of the distribution then becomes
$$
{\rm CDF}(\le x_{\rm med}) = 0.5 \quad ; \quad x_{\rm med} \equiv \text{the median value}.
$$
for the continous case. 

### Probability Distributions of Multiple Variables and Statistical Independence

In many cases that arise we are interested in the probability distribution of multiple random variables. For example, in the case of the American Community Survey, we might be interested in the **joint probability distribution** of household income and the number of people in the household $P(I,N)$. 

A related concept to a probability distribution of multiple variables, is a **conditional probability** distribution. The conditional probability is the probability that something occurs given that we know something else has occurred. For example, if we imagine holding the household size fixed at a specific value of $N$ (say $N=1$ person), what is the probability distribution of, $I$, the income for these one person households. You can imagine mapping this conditional income distribution out at each value of $N$. Mathematically, we write this as the conditional probability distribution $P(I|N)$, which is read as *the probability of $I$ given $N$*. 

In general, the joint probability distribution of two variables, $x$ and $y$, is related to the conditional probability by
<a id='eqn:P(x|y)'></a>
$$
P(x,y) = P(x|y)P(y), \tag{4} 
$$
which makes intuitive sense: the probability of $x$ and $y$ occurring together is the probability of $x$ occurring given that $y$ has occurred, times the probability of $y$ occurring on its own. Of course we could also think about it the opposite way, i.e.
<a id='eqn:P(y|x)'></a>
$$
P(x,y) = P(y|x)P(x), \tag{5}
$$
which says: the probability of $x$ and $y$ occurring together is the probability of $y$ occurring given that $x$ has occurred, times the probability of $x$ occurring on its own. 

The two variables $x$ and $y$ are **statistically independent** if the probability distribution of $x$ is not affected by the value of $y$, and vice versa, i.e. 
$$
P(x|y) = P(x) \quad \text{and} \quad P(y|x) = P(y).
$$
Plugging either of these equations into eqns.(<a href="#eqn:P(x|y)">4</a>) or (<a href="#eqn:P(y|x)">5</a>) we see that the joint probability distribution of $x$ and $y$ is the product of the individual probability distributions of $x$ and $y$, i.e.
<a id='eqn:joint'></a>
$$
P(x,y) = P(x)P(y). \tag{6}
$$
Again this makes intuitive sense. If we have two independent dice, the probability of rolling a 3 on one die is not affected by the value of the other die, and vice versa.  Hence the probability of rolling a 3 on both dice is the product of the probability of rolling a 3 on each die individually.

Finally, because of the symmetry of  eqns.(<a href="#eqn:P(x|y)">4</a>) or (<a href="#eqn:P(y|x)">5</a>), we can equate them to arrive at one of the most important results in probability theory, **Bayes' Theorem**:
$$
P(x,y) =  P(x|y)P(y) = P(y|x)P(x) \quad \rightarrow \quad P(x|y) = \frac{P(y|x)P(x)}{P(y)}, 
$$
which forms the foundation of **Bayesian statistics**. We will use Bayes' theorem in a future lecture when we discuss fitting models to data, also known as statistical inference. 

### Properties of the Mean and Variance Operators
The mean and variance over the probability distribution governing a random process are **operators** that instruct us to perform a specific mathematical operation. For example, consider a single continuous random variable $x$ governed by a probability distribution $P(x)$. For an arbitrary function $f(x)$, the mean over
the distribution is 
$$
\langle f(x) \rangle = \int f(x) P(x) dx, \quad \quad \text{mean; single variable $x$}. 
$$
and the variance is
$$
{\rm Var}[f(x)] = \int \left[f(x) - \langle f(x)\rangle\right]^2 P(x)dx, \quad \quad \text{variance; single variable $x$}.
$$
Similarly, for two random variables $x$ and $y$ governed by a joint probability distribution, $P(x,y)$, the mean of the function $f(x,y)$ would be
$$
\langle f(x,y) \rangle = \int f(x,y) P(x,y)dxdy, \quad \quad \text{mean; two variables $x$ and $y$},
$$
and  the variance is
$$
{\rm Var}[f(x,y)] = \int \left[f(x,y) - \langle f(x,y)\rangle\right]^2 P(x,y)dxdy, \quad \quad \text{variance; two variables $x$ and $y$}. 
$$


Later we will exploit the fact that these operators satisfy certain properties, *independent of the functional form of the probability distribution*.  For example if $A$ is a constant that does not depend on the random variable(s), then 
$$ 
\langle A \rangle = A \quad \text{and} \quad {\rm Var}(A) = 0, 
$$
which you can easily prove by using the definitions above.

Additionally, 
$$
\langle A x \rangle = A \langle x \rangle \quad \quad \text{and} \quad \quad {\rm Var}(A x) = A^2 {\rm Var}(x).   
$$
The mean is a **linear operator**, so for the two random variables $x$ and $y$, and constants $A$ and $B$, we have
$$
\langle Ax + By \rangle = A\langle x \rangle + B\langle y \rangle
$$

Finally, if $x$ and $y$ are **statistically independent** random variables, i.e. $P(x,y) = P(x)P(y)$, and $A$ and $B$ are constants, then it follows that
$$
 {\rm Var}(Ax + By) = A^2{\rm Var}(x) + B^2{\rm Var}(y), 
$$
where 
$$
\langle x \rangle = \int x P(x) dx, \quad \quad \text{and} \quad \quad {\rm Var}(x) = \int (x-\langle x\rangle)^2 P(x) dx. 
$$
and likewise for $y$.

You will prove these properties in the homework.